### Load packages

Before using those packages in gpu, be sure that all torch-related package is in correct cuda version.
- `nvidia-smi CUDA Version (12.5)`: This version indicates the maximum CUDA version supported by the installed NVIDIA driver. It reflects the driver’s capability to run CUDA applications that are built with any CUDA version up to 12.5.
- `nvcc --version CUDA Version (12.1)`: This version refers to the version of the CUDA toolkit installed on your system. The CUDA toolkit includes the CUDA compiler (nvcc), libraries, and other tools for developing CUDA applications.
- python == 3.10
- torch == 2.1.0+cu121 
- pyg_lib == 0.3.1+pt21cu121
- torch_cluster == 1.6.3+pt21cu121
- torch_scatter == 2.1.2+pt21cu121
- torch_sparse == 0.6.18+pt21cu121
- torch_spline_conv == 1.2.2+pt21cu121

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch.nn import Parameter

2.1.0+cu121
12.1
True


In [3]:
from torch_geometric.nn import GATConv, global_mean_pool
from torch_geometric.data import Data
from torch_geometric.utils import to_dense_adj, dense_to_sparse, degree, add_self_loops, remove_self_loops, softmax, is_torch_sparse_tensor
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.nn.inits import glorot, zeros
from torch_geometric.typing import Adj, OptTensor, PairTensor, OptPairTensor, Size, NoneType

In [4]:
from torch_sparse import SparseTensor, set_diag
import time

In [5]:
from torch_scatter import scatter_add
from typing import Optional, Tuple, Union
from livelossplot import PlotLosses
from pathlib import Path
from utility import MyGATConv, set_seed

In [6]:
import optuna
import gymnasium
# from .autonotebook import tqdm as notebook_tqdm
from ray import tune
from ray.tune import CLIReporter
from ray.tune import Trainable
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.optuna import OptunaSearch
# from ray.rllib.algorithms.ppo import PPO, PPOConfig

In [6]:
# import ray
# ray.init()

## rGAT Model:

****

- `radius_threshold`: 500~5000
- `interval`: the interval of each radius, setted as 50
- `brain_data`
- `select_family`
- **Other hyperparameters**: `my_hidden_channels`, `my_heads`,`my_dropout`,`my_lr`,`num_epochs`
- The graph attentional operator from the ["Graph Attention Networks"](https://arxiv.org/abs/1710.10903)


$$
        \mathbf{x}^{\prime}_i = \alpha_{i,i}\mathbf{\Theta}_{s}\mathbf{x}_{i} +
        \sum_{j \in \mathcal{N}(i)}
        \alpha_{i,j}\mathbf{\Theta}_{t}\mathbf{x}_{j},
$$


$$
        \alpha_{i,j} =
        \frac{
        \exp\left(\mathrm{LeakyReLU}\left(
        \mathbf{a}^{\top}_{s} \mathbf{\Theta}_{s}\mathbf{x}_i
        + \mathbf{a}^{\top}_{t} \mathbf{\Theta}_{t}\mathbf{x}_j
        \right)\right)}
        {\sum_{k \in \mathcal{N}(i) \cup \{ i \}}
        \exp\left(\mathrm{LeakyReLU}\left(
        \mathbf{a}^{\top}_{s} \mathbf{\Theta}_{s}\mathbf{x}_i
        + \mathbf{a}^{\top}_{t}\mathbf{\Theta}_{t}\mathbf{x}_k
        \right)\right)}.
$$

$$
        \alpha_{i,j} =
        \frac{
        \exp\left(\mathrm{LeakyReLU}\left(
        \mathbf{a}^{\top}_{s} \mathbf{\Theta}_{s}\mathbf{x}_i
        + \mathbf{a}^{\top}_{t} \mathbf{\Theta}_{t}\mathbf{x}_j
        + \mathbf{a}^{\top}_{e} \mathbf{\Theta}_{e} \mathbf{e}_{i,j}
        \right)\right)}
        {\sum_{k \in \mathcal{N}(i) \cup \{ i \}}
        \exp\left(\mathrm{LeakyReLU}\left(
        \mathbf{a}^{\top}_{s} \mathbf{\Theta}_{s}\mathbf{x}_i
        + \mathbf{a}^{\top}_{t} \mathbf{\Theta}_{t}\mathbf{x}_k
        + \mathbf{a}^{\top}_{e} \mathbf{\Theta}_{e} \mathbf{e}_{i,k}
        \right)\right)}.
$$

## Shapes:

- **input:**
  node features:
  $(|\mathcal{V}|, F_{in})$ or
  $((|\mathcal{V_s}|, F_{s}), (|\mathcal{V_t}|, F_{t}))$
  if bipartite,
  edge indices $(2, |\mathcal{E}|)$,
  edge features $(|\mathcal{E}|, D)$ *(optional)*
- **output:** node features $(|\mathcal{V}|, H * F_{out})$ or
  $((|\mathcal{V}_t|, H * F_{out})$ if bipartite.
  If `return_attention_weights=True`, then
  $((|\mathcal{V}|, H * F_{out}),
  ((2, |\mathcal{E}|), (|\mathcal{E}|, H)))$
  or $((|\mathcal{V_t}|, H * F_{out}), ((2, |\mathcal{E}|),
  (|\mathcal{E}|, H)))$ if bipartite



## Before Start... ##

****

- Check if all elements are stored in `cuda`
- Hyperparameter:
    - `my_hidden_channels`: 64
    - `my_heads`: 1
    - `dropout`: 0.6
    - `mu_lr`: learning rate set as 0.01
    - `epoch`: set epoch as 50, deter model from overfitting

### Set Head:

[Reference](https://petar-v.com/GAT/)

To stabilise the learning process of self-attention, we have found multi-head attention to be very beneficial [(as was the case in Vaswani et al., 2017)](https://arxiv.org/abs/1706.03762). Namely, the operations of the layer are independently replicated K times (each replica with different parameters), and outputs are featurewise aggregated (typically by concatenating or adding).
$$
\vec{h^\prime_i} = \Vert^k_{k=1} \sigma \lgroup \sum_{j\in N_i}\alpha^k_{i,j} \mathrm{W}^k \vec{h_j}\rgroup
$$
where $\alpha_{i,j}$ are the attention coefficients derived by the $k$-th replica, and $W_k$ the weight matrix specifying the linear transformation of the $k$-th replica. With the setup of the preceding sections, this fully specifies a **Graph Attention Network (GAT)** layer!

A GAT layer with multi-head attention. Every neighbour $i$ of node 1 sends its own vector of attentional coefficients,$\vec{\alpha_{1i}}$ one per each attention head $\alpha_{1i}^k$. These are used to compute K separate linear combinations of neighbours’ features $\vec{h_i}$, which are then aggregated (typically by concatenation or averaging) to obtain the next-level features of node 1,$\vec{h^\prime_1}$.

### Set Dropout:
Furthermore, we have found that applying dropout [(Srivastava et al., 2014)](https://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf) to the attentional coefficients $\alpha_{i,j}$ was a highly beneficial regulariser, especially for small training datasets. This effectively exposes nodes to stochastically sampled neighbourhoods during training, in a manner reminiscent of the (concurrently published) FastGCN method [(Chen et al., 2018)](https://arxiv.org/abs/1801.10247).

Dropout is a technique that addresses both these issues. It prevents overfitting and provides a way of approximately combining exponentially many different neural network
architectures efficiently. The term “dropout” refers to dropping out units (hidden and visible) in a neural network. By dropping a unit out, we mean temporarily removing it from the network, along with all its incoming and outgoing connections, as shown in Figure 1.

The choice of which units to drop is random. In the simplest case, each unit is retained with a fixed probability $p$ independent of other units, where $p$ can be chosen using a validation set or can simply be set at 0.5, which seems to be close to optimal for a wide range of networks and tasks. For the input units, however, the optimal probability of retention is usually closer to 1 than to 0.5

****

## K-Fold

The training data used in the model is split, into k number of smaller sets, to be used to validate the model. The model is then trained on k-1 folds of training set. The remaining fold is then used as a validation set to evaluate the model.

As we will be trying to classify different species of iris flowers we will need to import a classifier model, for this exercise we will be using a DecisionTreeClassifier. We will also need to import CV modules from sklearn.

```python
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, cross_val_score 
clf = DecisionTreeClassifier(random_state=42)
k_folds = KFold(n_splits = 5)
scores = cross_val_score(clf, X, y, cv = k_folds)

```

Assigning X and y:
- X (features): Typically, the x (node features) would be your input X. So in your case, X = data.x, which represents the feature matrix for the nodes.
- y (targets): The y here could refer to either:
- y (downstream genes): These could be your target variables if you are predicting gene expression or similar outcomes.
- labels (embedded cluster names): These could be your target labels if you are performing a clustering or classification task.

Which one to use as y depends on the specific task you are performing:
- If you are predicting gene expression (a multi-output task), use data.y.
- If you are performing node classification (predicting cluster labels), use data.labels.

### No finetune

In [7]:
# Define regular GAT model to get initial attention weights
class RegularGAT(torch.nn.Module):
    def __init__(self, in_channels_x, in_channels_t, out_channels, hidden_channels, heads, dropout):
        super(RegularGAT, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.embedding = torch.nn.Embedding(in_channels_t, hidden_channels).to(self.device)  # Embedding layer for T: cell_types
        self.gat1 = MyGATConv(in_channels_x, hidden_channels, heads=heads, dropout=0, add_self_loops=False)  # default dropout=0
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(hidden_channels + hidden_channels, hidden_channels),
            torch.nn.ReLU(),
            torch.nn.LayerNorm(hidden_channels), 
            torch.nn.Linear(hidden_channels, out_channels),
            torch.nn.ReLU(),
            torch.nn.LayerNorm(out_channels)
        ).to(self.device)
        self.bn = torch.nn.BatchNorm1d(out_channels * heads).to(self.device)
        self.dropout = dropout
        # self.gat2 = GATConv(hidden_channels * heads, out_channels, heads=1, concat=False, dropout=0)
        self.initial_attention_weights_abs = None
        self._initialize_weights()
    
    def forward(self, data):
        x, edge_index, t = data.x, data.edge_index, data.labels
        t_indices = torch.argmax(t, dim=1).to(self.device)
        t_emb = self.embedding(t_indices).to(self.device)  # Get the embedding for T
        x, (edge_index, (attention_scores, attention_weights)) = self.gat1(x, edge_index, return_attention_weights=True)
        print(f'after GAT relu x: {x}')
        
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = torch.cat([x, t_emb], dim=1)
        
        x = self.mlp(x)  # Apply MLP to combined features with LayerNorm
        x = F.dropout(x, p=self.dropout, training=self.training)
        print(f'after mlp and dropout x: {x}')
        
        # x, attention_weights = self.gat2(x, edge_index, return_attention_weights=True)
        
        if self.initial_attention_weights_abs is None:
            self.initial_attention_weights_abs = torch.abs(attention_weights[1]).detach()
        return x.to(self.device), attention_scores.to(self.device), attention_weights.to(self.device)  # Return the attention coefficients as well
   
    def _initialize_weights(self):
        for module in self.modules():
            if isinstance(module, torch.nn.Linear):
                torch.nn.init.xavier_uniform_(module.weight)

In [8]:
# Define training process
def train(data, device):
    model.train()
    data = data.to(device)
    optimizer.zero_grad()
    out, attention_scores, attention_weights = model(data)
    loss = F.mse_loss(out[data.train_mask], data.y[data.train_mask])
    # penalty = model.compute_attention_penalty(attention_weights)
    # total_loss = loss + penalty
    total_loss = loss
    total_loss.backward()
    optimizer.step()
    return total_loss.item(), attention_weights.cpu().detach().numpy(), attention_scores.cpu().detach().numpy()

def validate(data, device):
    model.eval()
    data = data.to(device)
    with torch.no_grad():
        out, attention_scores, attention_weights = model(data)
        loss = F.mse_loss(out[data.val_mask], data.y[data.val_mask])
        # penalty = model.compute_attention_penalty(attention_weights)
        # total_loss = loss + penalty
        total_loss = loss
    return total_loss.item(), attention_weights.cpu().detach().numpy(), attention_scores.cpu().detach().numpy()

def test(data, device):
    model.eval()
    data = data.to(device)
    with torch.no_grad():
        out, _, _ = model(data)
        test_loss = F.mse_loss(out[data.test_mask], data.y[data.test_mask])
    return test_loss.item()

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
torch.cuda.empty_cache()

cuda


In [10]:
file_path = './rGAT'
radius_threshold = 500  # 500, 200, 1000
brain_data = 'right'
select_family = 'Neurotrophins'

my_hidden_channels = 128
my_heads = 1
my_dropout = 0.3
my_lr = 0.001
num_epochs = 700

In [ ]:
data = torch.load(f'{file_path}/processed_for_CV/Radius_1000_Neurotrophins_ligand_target_left.data')

radius_threshold = 1000
model = RegularGAT(
    in_channels_x=data.x.shape[1], 
    in_channels_t=data.labels.shape[1], 
    out_channels=data.y.shape[1], 
    hidden_channels=my_hidden_channels, 
    heads=my_heads, 
    dropout=my_dropout
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=my_lr)

basename = f'RegularGAT_hidden_{my_hidden_channels}_head_{my_heads}_drop_{my_dropout}_lr_{my_lr}_right_Radius_200'

#####

checkpoint = f'{file_path}/rGAT_cosmx_yes_CV/models/checkpoints/{basename}'  # model weights
history = f'{file_path}/rGAT_cosmx_yes_CV/history/{basename}'  # attention score and weights
logfile = f'{file_path}/rGAT_cosmx_yes_CV/logs/log_{basename}.txt'  # training history

if not os.path.exists(checkpoint):
    os.makedirs(checkpoint)
if not os.path.exists(history):
    os.makedirs(history)


log_dir = os.path.dirname(logfile)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
if not os.path.exists(logfile):
    Path(logfile).touch()
    
log_file = open(logfile, 'w')

#####

train_losses = []
val_losses = []
attention_weights_history = [] ## after normalized
attention_scores_history = []

#####

logs = {}

groups = {'total_loss': ['train_total_loss', 'val_total_loss'],
          'sparsity': ['train_sparsity']}

for epoch in range(num_epochs):
    train_total_loss, train_attention_weights, train_attention_scores = train(data, device)
    val_total_loss, val_attention_weights, val_attention_scores = validate(data, device)
    logs['train_total_loss'] = train_total_loss
    logs['val_total_loss'] = val_total_loss
    log_file.write(f'{logs}\n')
    
    attention_weights_history.append(train_attention_weights)
    attention_scores_history.append(train_attention_scores)
    
    if (epoch + 1) % 500 == 0:
        torch.save(model.state_dict(), f'{checkpoint}/model_epoch_{epoch+1}.pth')
        
log_file.close()

np.save(history + '/attention_weights.npy', np.array(attention_weights_history))
test_loss = test(data, device)
print(f'Radius {radius_threshold}: Test MSE: {test_loss:.4f}')

### Supplementary: How NCEM Get R Square?

**Step1:**
- $y_{\text{true}}$ is the vector of true values.
- $y_{\text{pred}}$ is the vector of predicted values.

**Step2: The R-squared value,  $R^2$ , is calculated as:**
$$
R^2 = 1 - \frac{\sum_{i=1}^{n} \left( y_{\text{true}, i} - y_{\text{pred}, i} \right)^2}{\sum_{i=1}^{n} \left( y_{\text{true}, i} - \bar{y}_{\text{true}} \right)^2}
$$

$$
\bar{y}_{\text{true}} = \frac{1}{n} \sum_{i=1}^{n} y_{\text{true}, i}
$$

**Explanation**

- Residual Sum of Squares (RSS):  $\sum_{i=1}^{n} \left( y_{\text{true}, i} - y_{\text{pred}, i} \right)^2$
- Total Sum of Squares (TSS):  $\sum_{i=1}^{n} \left( y_{\text{true}, i} - \bar{y}_{\text{true}} \right)^2$

In [ ]:
def r_squared(y_true, y_pred):
    """Compute custom r squared.

    Parameters
    ----------
    y_true
        y_true.
    y_pred
        y_pred.

    Returns
    -------
    r2
    """
    print('y_pred is:',y_pred)
    
    y_pred, _ = tf.split(y_pred, num_or_size_splits=2, axis=2)
    # print('after processed y_pred is: \n',y_pred)
    # print('what is splited _: \n',_)
    
    residual = tf.reduce_sum(tf.square(y_true - y_pred))
    # print('residual is? \n',residual)
    
    total = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    # print('what is total: \n',total)
    r2 = tf.subtract(1.0, tf.math.divide(residual, total))
    print(r2)
    return r2

In [ ]:
r2_value = r_squared(y_true_tf, y_pred_tf)